# 事前準備

In [1]:
from keras.datasets import reuters

(x_train, y_train), (x_test, y_test) = reuters.load_data(path="reuters.npz",
                                                         num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=113,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

2121728/2110848 [==============================] - 0s 0us/step


In [7]:
word_index = reuters.get_word_index(path="reuters_word_index.npz")

565248/550378 [==============================] - 0s 0us/step


## データについて
xのデータには文字に対応するindexが入っている。indexとの対応はword_indexに文字を打ち込むとindexが得られる辞書がある。  
元の文章を得るにはindexから文字を取得できる様にする必要がある。  
また、yにはトピックのラベルが入っている。この番号を予想するのが目的となる。

In [14]:
# バッチ35の10個目の単語を求める
x_train[35][10]

343

In [32]:
# word_index.items()でkey(文字)とvalues(数字、上で得られたもの)の並びを取得できる
# 新しくindexを数字として、文字を引ける様に設定する
# index_from=3で書いたので、3から始まるようにする
index_word_map = {index + 3: word for word, index in word_index.items()}

In [38]:
# また、　index0, 1, 2に該当する意味をしていする。
index_word_map[0] = "[padding]"
index_word_map[1] = "[start]"

# OOVはout of vocabularyの略
index_word_map[2] = "[oov]"

In [36]:
index_word_map[343]

'make'

In [37]:
for index in x_train[0]:
    print(index_word_map[index], end = " ")

[start] mcgrath rentcorp said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3 

mcgrath rentcorpは、12月にspace coを買収した結果、1987年の1株当たり利益は1986年の70ctsから115から130 dlrsに増加すると見込んでおり、税引前純額は6から9から1000万dlrsに上昇すると述べた。 1986年のmlndlrsと、12 5 mlndlrsから19から22mln dlrsへのレンタル運用収益は、今年の1株あたりのキャッシュフローは250から3dlrs reuter3であると述べています。  
おそらくできてる。

## 分析可能な形に前処理

In [48]:
# 頻出1000の文字に限定する
(x_train, y_train), (x_test, y_test) = reuters.load_data(path="reuters.npz",
                                                         num_words=1000,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=113,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [49]:
# oovのものは1000位以下の文字になる。
for index in x_train[0]:
    print(index_word_map[index], end = " ")

[start] [oov] [oov] said as a result of its december acquisition of [oov] co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and [oov] [oov] revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash [oov] per share this year should be 2 50 to three dlrs reuter 3 

In [52]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 上位1000個をベクトルにする
tokenizer = Tokenizer(num_words=1000)

tokenizer.fit_on_sequences(x_train)

# tfidfにしてみる
x_train_tfidf = tokenizer.sequences_to_matrix(x_train, "tfidf")
x_test_tfidf = tokenizer.sequences_to_matrix(x_test, "tfidf")

In [54]:
x_train_tfidf.shape

(8982, 1000)

In [56]:
x_test_tfidf[0][:5]

array([0.        , 0.69309152, 2.84159064, 0.        , 2.65863303])

## 目的変数をOnehot

In [43]:
y_train[0]

3

In [45]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

In [47]:
y_test_one_hot.shape

(2246, 46)

# 【問題1】各種手法の実行
Kerasには4種類のReccurentレイヤーが用意されています。SimpleRNN以外はゲート付きリカレントニューラルネットワークです。


SimpleRNN
GRU
LSTM
ConvLSTM2D

これらを実行してください。この中でSimpleRNN、GRU、LSTMは同様のタスクに用いることができるため、精度の比較も行なってください。


Keras公式のサンプルコードを利用してください。

# 【問題2】（アドバンス課題）複数のデータセット間での比較
他のデータセットでも実験を行なってください。


データセット - Keras Documentation


Kerasで簡単に利用できる自然言語データセットとしてロイターのニュースワイヤー トピックス分類があります。

## 試しにRNNを使わない場合

In [75]:
import tensorflow as tf
import keras as keras

model = tf.keras.models.Sequential([
layers.Dense(512, input_shape=(1000,), activation='relu'),
layers.Dense(y_train_one_hot.shape[1], activation='softmax')
])

In [76]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               512512    
_________________________________________________________________
dense_8 (Dense)              (None, 46)                23598     
Total params: 536,110
Trainable params: 536,110
Non-trainable params: 0
_________________________________________________________________


In [77]:
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics='accuracy'
)

In [78]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(x_train_tfidf, y_train_one_hot, epochs=100, validation_split=0.2, callbacks=[early_stop])

Epoch 1/100
225/225 [==============================] - ETA: 54s - loss: 4.5475 - accuracy: 0.0000e+0 - ETA: 0s - loss: 2.1991 - accuracy: 0.5521    - ETA: 0s - loss: 1.7755 - accuracy: 0.63 - ETA: 0s - loss: 1.5552 - accuracy: 0.67 - ETA: 0s - loss: 1.4562 - accuracy: 0.70 - ETA: 0s - loss: 1.3471 - accuracy: 0.72 - ETA: 0s - loss: 1.2687 - accuracy: 0.73 - ETA: 0s - loss: 1.2366 - accuracy: 0.74 - ETA: 0s - loss: 1.2011 - accuracy: 0.74 - 1s 3ms/step - loss: 1.1936 - accuracy: 0.7491 - val_loss: 0.8883 - val_accuracy: 0.8052
Epoch 2/100
225/225 [==============================] - ETA: 0s - loss: 0.4616 - accuracy: 0.87 - ETA: 0s - loss: 0.4047 - accuracy: 0.90 - ETA: 0s - loss: 0.3799 - accuracy: 0.90 - ETA: 0s - loss: 0.3794 - accuracy: 0.90 - ETA: 0s - loss: 0.3921 - accuracy: 0.90 - ETA: 0s - loss: 0.3923 - accuracy: 0.90 - ETA: 0s - loss: 0.4045 - accuracy: 0.90 - ETA: 0s - loss: 0.4076 - accuracy: 0.90 - ETA: 0s - loss: 0.4167 - accuracy: 0.90 - 1s 2ms/step - loss: 0.4209 - accura

In [79]:
import pandas as pd
pd.DataFrame(history.history)

,loss,accuracy,val_loss,val_accuracy
0,1.193644,0.749061,0.888307,0.805231
1,0.420866,0.904245,0.864388,0.809683
2,0.273012,0.938900,0.912139,0.815248
3,0.211553,0.952958,0.942858,0.810796
4,0.195646,0.957550,0.916488,0.809126
5,0.179001,0.957550,0.972237,0.801336
6,0.166737,0.959638,0.939287,0.796884


In [80]:
model.evaluate(x_test_tfidf, y_test_one_hot)

71/71 [==============================] - ETA: 1s - loss: 0.5248 - accuracy: 0.87 - ETA: 0s - loss: 0.9763 - accuracy: 0.78 - 0s 1ms/step - loss: 1.0158 - accuracy: 0.7858


[1.0157599449157715, 0.7858415246009827]

79％の正解率となった。  
RNNを使うと上がるのか下がるのか...

## SimpleRNN

In [121]:
model = tf.keras.models.Sequential([
    layers.SimpleRNN(512, activation='tanh', input_shape=(None, 1000)),
    layers.Dense(y_train_one_hot.shape[1], activation='softmax')
])

ちなみに  
model.add(Embedding(max_features, 1000))  
にすることで、前処理の同等の結果が得られる。

In [122]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_5 (SimpleRNN)     (None, 512)               774656    
_________________________________________________________________
dense_14 (Dense)             (None, 46)                23598     
Total params: 798,254
Trainable params: 798,254
Non-trainable params: 0
_________________________________________________________________


In [123]:
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics='accuracy'
)

In [124]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(x_train_tfidf, y_train_one_hot, epochs=100, validation_split=0.2, callbacks=[early_stop])

Epoch 1/100


ValueError: in user code:

    /opt/anaconda3/envs/always/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /opt/anaconda3/envs/always/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/anaconda3/envs/always/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/anaconda3/envs/always/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/anaconda3/envs/always/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/anaconda3/envs/always/lib/python3.8/site-packages/keras/engine/training.py:

??  
shapeを三次元にしてくださいとのこと....  
シーケンス方向にデータを拡張する必要があるのかな...  
今回の場合文字列なので(バッチ、 1, tfidf)の形???  
あるいは1000個の文字数なので、　(batch, 登場する単語数, テキストの長さ)にすることで、RNNにできそう

In [133]:
# 500個の文字ベクトルにする。空いてるところは0をいれちゃう
x_train_500 = sequence.pad_sequences(x_train, maxlen=500)

In [134]:
x_train_500.shape

(8982, 500)

In [140]:
test = tf.keras.models.Sequential([
    layers.Embedding(1000, 500),
    layers.SimpleRNN(512, activation='tanh', input_shape=(1000, 500)),
    layers.Dense(y_train_one_hot.shape[1], activation='softmax')
])

In [141]:
test.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics='accuracy'
)

In [146]:
test.fit(x_train_500, y_train_one_hot, epochs=5, verbose=2)

Epoch 1/5
281/281 - 189s - loss: 2.3067 - accuracy: 0.4092
Epoch 2/5
281/281 - 185s - loss: 2.2459 - accuracy: 0.4241
Epoch 3/5
281/281 - 182s - loss: 2.1797 - accuracy: 0.4444
Epoch 4/5
281/281 - 183s - loss: 2.1528 - accuracy: 0.4574
Epoch 5/5
281/281 - 184s - loss: 2.1288 - accuracy: 0.4653


In [147]:
test.evaluate(x_test_tfidf, y_test_one_hot, verbose=2)

71/71 - 26s - loss: 2.6955 - accuracy: 0.3615


[2.6955084800720215, 0.3615316152572632]

一応学習はできたが、精度は低め...  
パラメータな気もする...  
他人のコード見てみる  

- 単語数は全部で1000  
- 文章の長さは500,もしかしたらここかも  
- 学習率？

In [151]:
x_train_250 = sequence.pad_sequences(x_train, maxlen=250)

In [152]:
test = tf.keras.models.Sequential([
    layers.Embedding(1000, 250),
    layers.SimpleRNN(512, activation='tanh', input_shape=(1000, 250)),
    layers.Dense(y_train_one_hot.shape[1], activation='softmax')
])

In [153]:
test.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics='accuracy'
)

In [154]:
test.fit(x_train_250, y_train_one_hot, epochs=5, verbose=2)

Epoch 1/5
281/281 - 77s - loss: 2.5751 - accuracy: 0.3252
Epoch 2/5
281/281 - 78s - loss: 2.3808 - accuracy: 0.3805
Epoch 3/5
281/281 - 75s - loss: 2.1904 - accuracy: 0.4305
Epoch 4/5
281/281 - 71s - loss: 2.0926 - accuracy: 0.4579
Epoch 5/5
281/281 - 75s - loss: 2.0059 - accuracy: 0.4773


In [156]:
x_test_250 = sequence.pad_sequences(x_test, maxlen=250)

In [158]:
test.evaluate(x_test_250, y_test_one_hot, verbose=2)

71/71 - 5s - loss: 2.2729 - accuracy: 0.4029


[2.272853136062622, 0.4029385447502136]

精度が上がった,x=0のデータが多いと重みが小さくなるよねっていう話だと思う。  
epoch増やしてみる

In [159]:
test = tf.keras.models.Sequential([
    layers.Embedding(1000, 250),
    layers.SimpleRNN(512, activation='tanh', input_shape=(1000, 250)),
    layers.Dense(y_train_one_hot.shape[1], activation='softmax')
])

In [160]:
test.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics='accuracy'
)

In [161]:
test.fit(x_train_250, y_train_one_hot, epochs=10, verbose=2)

Epoch 1/10
281/281 - 78s - loss: 2.8789 - accuracy: 0.2943
Epoch 2/10
281/281 - 71s - loss: 2.5170 - accuracy: 0.3434
Epoch 3/10
281/281 - 70s - loss: 2.7947 - accuracy: 0.3007
Epoch 4/10
281/281 - 71s - loss: 2.4495 - accuracy: 0.3573
Epoch 5/10
281/281 - 71s - loss: 2.4716 - accuracy: 0.3435
Epoch 6/10
281/281 - 76s - loss: 2.4439 - accuracy: 0.3545
Epoch 7/10
281/281 - 75s - loss: 2.4705 - accuracy: 0.3420
Epoch 8/10
281/281 - 73s - loss: 2.3531 - accuracy: 0.3863
Epoch 9/10
281/281 - 71s - loss: 2.2712 - accuracy: 0.4187
Epoch 10/10
281/281 - 71s - loss: 2.2143 - accuracy: 0.4328


In [163]:
test.evaluate(x_test_250, y_test_one_hot, verbose=2)

71/71 - 5s - loss: 2.2523 - accuracy: 0.4412


[2.252274990081787, 0.44122883677482605]

？？  
epoch増やしただけなのに学習経過がだいぶ変更された。  
入力データの順番が変わったのだと思うが、ここまで変わるのか・

## LSTM

![Imgur](https://i.imgur.com/uWwm9lRh.png)

直近の過去の重みhといままでの過去の重みcの二つの重みが存在している。  


1. hとxを足して、記憶セルcにどれくらい書き込むかをシグモイド関数によって確定させる。  
    その後記憶せるcに書き込む
    ![Imgur](https://i.imgur.com/Jwxfqhrh.jpg)
    
2. 1で得られたものと、RNNのtanhで活性化させたものを掛け合わせる。
    イメージ的にはhとxが関連してるベクトルならシグモイド関数の出力は大きくなるので、データの価値は大きくなる。 
    ![Imgur](https://i.imgur.com/g6RwjL3h.jpg)
    
3. 古いパラメータを更新する。まず1で求めた現時点の情報をかけて、古い情報を消す。  
    そこに、2で求めた新しいデータを足し合わせる。  
    ![Imgur](https://i.imgur.com/1mnqlqzh.jpg)
    
4. 更新されたcにtanhを掛け合わせたものと、今回だけのデータのシグモイド関数にかけたものを掛け合わせる。
    ![Imgur](https://i.imgur.com/IepA3aCh.jpg)

In [164]:
lstm = tf.keras.models.Sequential([
    layers.Embedding(1000, 250),
    layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(y_train_one_hot.shape[1], activation='softmax')
])

In [167]:
lstm.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics='accuracy'
)

In [168]:
lstm.fit(x_train_250, y_train_one_hot, epochs=10, verbose=2)

Epoch 1/10
281/281 - 98s - loss: 2.1789 - accuracy: 0.4614
Epoch 2/10
281/281 - 99s - loss: 1.8635 - accuracy: 0.5278
Epoch 3/10
281/281 - 96s - loss: 1.6386 - accuracy: 0.5825
Epoch 4/10
281/281 - 92s - loss: 1.4439 - accuracy: 0.6420
Epoch 5/10
281/281 - 104s - loss: 1.3318 - accuracy: 0.6668
Epoch 6/10
281/281 - 103s - loss: 1.2037 - accuracy: 0.7057
Epoch 7/10
281/281 - 104s - loss: 1.1091 - accuracy: 0.7281
Epoch 8/10
281/281 - 98s - loss: 0.9866 - accuracy: 0.7604
Epoch 9/10
281/281 - 96s - loss: 0.8875 - accuracy: 0.7825
Epoch 10/10
281/281 - 94s - loss: 0.8153 - accuracy: 0.8010


In [171]:
lstm.evaluate(x_test_250, y_test_one_hot, verbose=2)

71/71 - 3s - loss: 1.0306 - accuracy: 0.7489


[1.0306174755096436, 0.7488868832588196]

精度高い、まだ学ばせれば精度上がりそう。　  
計算コストは30%くらい増えてる。

## GRU

計算コストがLSTMより小さい、性能はそこまで変わらない。
![Imgur](https://i.imgur.com/X2yC7dzh.png)

In [172]:
gru = tf.keras.models.Sequential([
    layers.Embedding(1000, 250),
    layers.GRU(128, dropout=0.2, recurrent_dropout=0.2),
    layers.Dense(y_train_one_hot.shape[1], activation='softmax')
])

In [173]:
gru.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics='accuracy'
)

In [174]:
gru.fit(x_train_250, y_train_one_hot, epochs=10, verbose=2)

Epoch 1/10
281/281 - 87s - loss: 2.0948 - accuracy: 0.4566
Epoch 2/10
281/281 - 85s - loss: 1.6594 - accuracy: 0.5772
Epoch 3/10
281/281 - 85s - loss: 1.5012 - accuracy: 0.6306
Epoch 4/10
281/281 - 87s - loss: 1.2827 - accuracy: 0.6847
Epoch 5/10
281/281 - 97s - loss: 1.1890 - accuracy: 0.7050
Epoch 6/10
281/281 - 97s - loss: 1.0668 - accuracy: 0.7346
Epoch 7/10
281/281 - 86s - loss: 0.9357 - accuracy: 0.7690
Epoch 8/10
281/281 - 89s - loss: 0.8833 - accuracy: 0.7837
Epoch 9/10
281/281 - 88s - loss: 0.7999 - accuracy: 0.8010
Epoch 10/10
281/281 - 92s - loss: 0.7457 - accuracy: 0.8130


In [175]:
gru.evaluate(x_test_250, y_test_one_hot, verbose=2)

71/71 - 3s - loss: 1.0156 - accuracy: 0.7560


[1.01558518409729, 0.7560107111930847]

性能はLSTMとそこまで大きくかわらない  
実行時間は-10%くらい

## ConvLSTM2D

LSTMの行列計算で行っていたのを2Dに拡張し、アダマール積でLSTMを行うもの。  
画像に時系列の重みを持たせられるので、天気予報とかに使えそう。  
ただ、計算コストは高そう。

# 【問題3】他のクラスの説明
ドキュメントには他にも関連するクラスが記載されています。それらがどういうものなのかを説明してください。この中には実際に扱うことは少ないクラスも含まれています。

- RNN
    セル単位でRNNを行うための関数  
    
- SimpleRNNCell
- GRUCell
- LSTMCell
    RNNをセル単位で行うための関数。行うことはcellじゃないのと同じ。
- StackedRNNCells  
    複数のRNNセルを単一のセルの様に見せる関数
    
- CuDNNGRU
- CuDNNLSTM
    GPU環境でGPU,LSTMを行うためのライブラリ

# 考察

RNNの手法の流れは理解できた。  
基本的にはLSTM, GRUを両方ためして精度が変わらないならGRUメインでチューニングを行うのがいいと考えられる。

# 疑問
問題3で述べたセル単位でRNN手法を変えるやり方がどれくらい効果的なのかイメージがつきづらかった。  
おそらくだが、SimpleRnn→LSTMといった様に同シーケンスで2回重みを学習させることができると考えられる。  
計算コストは増えるが、同じシーケンスで学習ができるので出力はより慎重な値になるのではないかと考えれる。